<a href="https://colab.research.google.com/github/saheedniyi02/fpl-ai/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
gameweek=16


import pandas as pd
import numpy as np
import warnings
import os
from torch.utils.tensorboard import SummaryWriter
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch
from utils import *


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)

from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

def replace_spaces_with_underscore(column_list):
    return [col.replace(' ', '_') for col in column_list]

# Applying the transformation to the existing lists

from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

class LossCallback:
    def __init__(self, log_dir):
        self.log_dir = log_dir
        self.writer = SummaryWriter(log_dir=log_dir)

    def __call__(self, epoch, iteration, loss):
        global_step = epoch * len(train_loader) + iteration
        self.writer.add_scalar("Training Loss", loss.item(), global_step=global_step)

    def close(self):
        self.writer.close()
        
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3',"mean assists all",
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


forward_statistics = replace_spaces_with_underscore(forward_statistics)
midfielder_statistics = replace_spaces_with_underscore(midfielder_statistics)
goalkeeper_statistics = replace_spaces_with_underscore(goalkeeper_statistics)
statistics = replace_spaces_with_underscore(statistics)

# statistics = replace_spaces_with_underscore(statistics)

date_cols=["day_of_week","month","hour","week"]

path = f"predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)




# x, val, y, y_val = train_test_split(
#     train.drop(leak_columns, axis=1),
#     target["minutes"],
#     test_size=0.1,
#     random_state=0,
# )


Folder predicted_dataset/GW16 created!


In [16]:
# print(train.keys())

# Minutes

In [17]:
train = pd.read_csv("cleaned_dataset/cleaned_previous_seasons.csv",header=0,index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
# train.columns = train.columns.str.strip()
train.columns = replace_spaces_with_underscore(train.columns.tolist())

old_gameweek_cleaned = []
for i in range(1, gameweek):
    # old_gameweek_cleaned.append(pd.read_csv(f"cleaned_dataset/2023-24/GW{i}.csv"))
    df = pd.read_csv(f"cleaned_dataset/2023-24/GW{i}.csv")
    # Replace spaces with underscores in column names
    df.columns = df.columns.str.replace(' ', '_')
    old_gameweek_cleaned.append(df)

old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"cleaned_dataset/2023-24/GW{gameweek}.csv", header=0,index_col=0)
# test.columns = test.columns.str.strip()
test.columns = replace_spaces_with_underscore(test.columns.tolist())

from sklearn.preprocessing import LabelEncoder

def filter_existing_columns(df, columns):
    return [col for col in columns if col in df.columns]

train["position"]=train["position"].replace({"GKP":"GK" })
test["position"]=test["position"].replace({"GKP":"GK" })

test = test[train.columns]
train_copy = train.copy()
test_copy = test.copy()

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
train["date"]=pd.to_datetime(train["kickoff_time"])
train["day_of week"]=train["date"].dt.day_name
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
train["week"]=train["date"].dt.week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test= test.set_index("index")
test["date"]=pd.to_datetime(test["kickoff_time"])
test["day_of week"]=test["date"].dt.day_name
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
test["week"]=test["date"].dt.week
test.drop(["kickoff_time","date"], axis=1, inplace=True)
train["minutes"] = train["minutes"].apply(convert_minutes)

target = train[["minutes", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)

for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

# train = train[statistics + leak_columns+date_cols]
# test = test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["minutes"],
    test_size=0.1,
    random_state=0,
)

params={'colsample_bylevel': 0.8070621518153563, 'learning_rate': 0.04765984972709895, 'max_depth': 7, 'reg_lambda': 5, 'scale_pos_weight': 2.5,'subsample': 0.6794390204583894}
model=CatBoostClassifier(**params,cat_features=["position"],random_state=0,early_stopping_rounds=500,use_best_model=True,verbose=500,n_estimators=10000)

model.fit(x, y,eval_set=[(val,y_val)])

# test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))
test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))

test_copy[leak_columns + ["minutes"]].to_csv(
    f"minutes.csv"
)

predicted_minutes=model.predict(val)
val_=pd.DataFrame({"ind":val.index,"actul_minutes":y_val,"predicted_minutes":predicted_minutes,"position":val["position"]})

train_copy0 = train_copy
test_copy0 = test_copy

0:	learn: 0.5901719	test: 0.5902151	best: 0.5902151 (0)	total: 50.7ms	remaining: 8m 26s
500:	learn: 0.0342086	test: 0.0392857	best: 0.0392857 (499)	total: 21.6s	remaining: 6m 50s
1000:	learn: 0.0280242	test: 0.0389889	best: 0.0388980 (919)	total: 43.3s	remaining: 6m 28s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.03889800562
bestIteration = 919

Shrink model to first 920 iterations.


In [18]:
# print(test_copy0)
# print(np.sum(test_copy['minutes']==1))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torch.cuda.is_available()
# print(device)

# Points

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error

%env CUDA_VISIBLE_DEVICES=0


train = pd.read_csv("cleaned_dataset/cleaned_previous_seasons.csv",header=0,index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
# train.columns = train.columns.str.strip()
train.columns = replace_spaces_with_underscore(train.columns.tolist())

old_gameweek_cleaned = []
for i in range(1, gameweek):
    # old_gameweek_cleaned.append(pd.read_csv(f"cleaned_dataset/2023-24/GW{i}.csv"))
    df = pd.read_csv(f"cleaned_dataset/2023-24/GW{i}.csv")
    # Replace spaces with underscores in column names
    df.columns = df.columns.str.replace(' ', '_')
    old_gameweek_cleaned.append(df)

old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"cleaned_dataset/2023-24/GW{gameweek}.csv", header=0,index_col=0)
# test.columns = test.columns.str.strip()
test.columns = replace_spaces_with_underscore(test.columns.tolist())

from sklearn.preprocessing import LabelEncoder

def filter_existing_columns(df, columns):
    return [col for col in columns if col in df.columns]

train["position"]=train["position"].replace({"GKP":"GK" })
test["position"]=test["position"].replace({"GKP":"GK" })

# train = train_copy[train_copy0["minutes"] > 0]
# test = test_copy[test_copy0["minutes"] > 0]

test = test[train.columns]
train_copy = train.copy()
test_copy = test.copy()

# predict points
train["index"] = train["name"] + train['kickoff_time'].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
train["date"]=pd.to_datetime(train['kickoff_time'])
train["day_of_week"]=train["date"].dt.day_name
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
train["week"]=train["date"].dt.week
train.drop(['kickoff_time',"date"], axis=1, inplace=True)

test["index"] = test["name"] + test['kickoff_time'].astype("str")
test = test.set_index("index")
test["date"]=pd.to_datetime(test['kickoff_time'])
test["day_of_week"]=test["date"].dt.day_name
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
test["week"]=test["date"].dt.week
test.drop(['kickoff_time',"date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]

largest_gw = find_largest_gw('datasets/2023-24/fixtures/')
if largest_gw > gameweek:
    test_target = test[["total_points", "GW","position" ]]
    

train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})


selected_columns = statistics + leak_columns + date_cols
selected_columns_filtered = filter_existing_columns(train, selected_columns)
train = train[selected_columns_filtered]
test = test[selected_columns_filtered]

train = train.fillna(0)
test = test.fillna(0)

label_encoder_name = LabelEncoder()
combined_data = pd.concat([train, test], axis=0)
combined_data['name'] = combined_data['name'].astype(str)
combined_data['name_encoded'] = label_encoder_name.fit_transform(combined_data['name'])

label_encoder_team = LabelEncoder()
combined_data['team_encoded'] = label_encoder_team.fit_transform(combined_data['team'])

label_encoder_position = LabelEncoder()
combined_data['position'] = label_encoder_position.fit_transform(combined_data['position'])


# List of non-numeric columns
non_numeric_cols = ['was_home', 'name', 'position', 'team']

for col in non_numeric_cols:
    if col == 'name':
        train[col] = label_encoder_name.transform(train[col])
        test[col] = label_encoder_name.transform(test[col])
    elif col == 'team':
        train[col] = label_encoder_team.transform(train[col])
        test[col] = label_encoder_team.transform(test[col])
    elif col == 'position':
        train[col] = label_encoder_position.transform(train[col])
        test[col] = label_encoder_position.transform(test[col])


name_mapping = dict(zip(label_encoder_name.classes_, label_encoder_name.transform(label_encoder_name.classes_)))
team_mapping = dict(zip(label_encoder_team.classes_, label_encoder_team.transform(label_encoder_team.classes_)))
position_mapping = dict(zip(label_encoder_position.classes_, label_encoder_position.transform(label_encoder_position.classes_)))

# Invert the dictionaries to map back from encoded values to original strings
name_id_to_string = {v: k for k, v in name_mapping.items()}
team_id_to_string = {v: k for k, v in team_mapping.items()}
position_id_to_string = {v: k for k, v in position_mapping.items()}

# train = train_copy[train_copy0["minutes"] > 0]
# test = test_copy[test_copy0["minutes"] > 0]

# Define a custom PyTorch Dataset
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.targets = torch.tensor(targets.values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

# Create datasets and data loaders
train_dataset = CustomDataset(train, target["total_points"])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)




import torch.nn as nn
import torch.nn.functional as F




from torch.utils.tensorboard import SummaryWriter

class LossCallback:
    def __init__(self, log_dir):
        self.log_dir = log_dir
        self.writer = SummaryWriter(log_dir=log_dir)
        self.update_frequency = 1000
        self.running_loss = 0.0

    def __call__(self, epoch, iteration, loss):
        global_step = epoch * len(train_loader) + iteration
        self.running_loss += loss.item()

        if iteration % self.update_frequency == (self.update_frequency - 1):  # Log every 100 iterations
            average_loss = self.running_loss / self.update_frequency
            self.writer.add_scalar("Training Loss", average_loss, global_step=global_step)
            self.running_loss = 0.0  # Reset running loss

    def close(self):
        self.writer.close()

input_size = train.shape[1]  # Update this with your actual input size
hidden_size = 64  # Define the size of the LSTM hidden state

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

lambda_value = 0.001 

# Define the Lasso regression model
class LassoRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LassoRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

# Instantiate the Lasso regression model
input_size = train_dataset.features.shape[1]  # Assuming the features shape is known
lasso_model = LassoRegressionModel(input_size)
lasso_model = lasso_model.to(device)  # Move the model to CUDA if available

# Define L1 loss (Lasso regression uses L1 loss)
criterion = nn.L1Loss()

# Define optimizer (e.g., Adam optimizer)
optimizer = optim.Adam(lasso_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    lasso_model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = lasso_model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))  # Ensure targets have shape [batch_size, 1]
        
        # L1 regularization term (sum of absolute values of model parameters)
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        for param in lasso_model.parameters():
            l1_reg += torch.norm(param, 1)
        
        loss += lambda_value * l1_reg  # lambda_value is the regularization strength
        
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

# Evaluate your model on the test set if available
# Remember to use the same preprocessing steps on the test set and evaluate the trained model


env: CUDA_VISIBLE_DEVICES=0
Epoch [1/10], Loss: 1.1851069927215576
Epoch [2/10], Loss: 1.63762629032135
Epoch [3/10], Loss: 0.11007190495729446
Epoch [4/10], Loss: 0.36771464347839355
Epoch [5/10], Loss: 0.7295762300491333
Epoch [6/10], Loss: 0.8657636046409607
Epoch [7/10], Loss: 0.3829452395439148
Epoch [8/10], Loss: 8.8289155960083
Epoch [9/10], Loss: 3.5651440620422363
Epoch [10/10], Loss: 0.24317751824855804


In [55]:
# print(train.shape,target["total_points"].shape)
# print(target["total_points"].shape)
print(largest_gw,gameweek)

18 16


In [58]:
# Assuming your test data is preprocessed similarly to the training data

if largest_gw > gameweek:
    test_inputs = torch.tensor(test.values, dtype=torch.float32)  # Assuming test data is in a pandas DataFrame
    test_targets = torch.tensor(test_target["total_points"].values, dtype=torch.float32)  # Assuming test targets are in a DataFrame
else:
    test_inputs = torch.tensor(test.values, dtype=torch.float32)  # Assuming test data is in a pandas DataFrame
    test_targets = torch.zeros((test.shape[0],), dtype=torch.float32)  # Creating zero targets for the test set

# Ensure data is moved to the device
test_inputs = test_inputs.to(device)
test_targets = test_targets.to(device)

lasso_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = lasso_model(test_inputs)
    test_outputs = test_outputs.view(-1)  # Adjust shape if needed


from sklearn.metrics import mean_squared_error

test_predictions = test_outputs.cpu().numpy()  # Move predictions to CPU and convert to numpy
test_targets = test_targets.cpu().numpy()  # Move targets to CPU and convert to numpy

performance = mean_squared_error(test_targets, test_predictions)
print(f"Mean Squared Error (MSE) on Test Set: {mse}")


Mean Squared Error (MSE) on Test Set: 12.014159202575684


In [59]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error

file_path = 'performance.csv'
if not os.path.exists(file_path):
    initial_data = {'lgbm': [0], 'lasso': [0], 'cnn': [0]}
    pd.DataFrame(initial_data).to_csv(file_path, index=False)

predictions_df2 = pd.read_csv(file_path)
for i in range(predictions_df2.index.max() + 1, gameweek+1):
    if i not in predictions_df2.index:
        predictions_df2.loc[i] = [0, 0, 0]
    else:
        missing_columns = set(['lgbm', 'lasso', 'cnn']) - set(predictions_df2.columns)
        for col in missing_columns:
            predictions_df2.loc[i, col] = 0

# performance = mean_squared_error(preds, test_target['total_points'])
if 'lasso' in predictions_df2.columns:
    if pd.isnull(predictions_df2.loc[gameweek, 'lasso']) or predictions_df2.loc[gameweek, 'lasso']==0:
        predictions_df2.loc[gameweek-1, 'lasso'] = performance
    else:
        predictions_df2.loc[gameweek-1, 'lasso'] += performance
        
predictions_df2.to_csv(file_path, index=False)


In [67]:
test_predictions.shape

(673,)

In [68]:

import numpy as np
import pandas as pd

# Combine the predictions with the test data
test["points"] = test_predictions

# Assuming leak_columns is a list of columns to include in the CSV file
columns_to_include = leak_columns + ["points", "value"]


test['name'] = test['name'].map(name_id_to_string)
test['team'] = test['team'].map(team_id_to_string)
test['position'] = test['position'].map(position_id_to_string)


2.7320675301713573
1.6917027774360753

In [69]:

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [70]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Mohamed Salah2023-12-09T12:30:00Z,Mohamed Salah,5.895228,Liverpool
Marcus Rashford2023-12-09T15:00:00Z,Marcus Rashford,5.002692,Man Utd
Miguel Almirón Rejala2023-12-10T16:30:00Z,Miguel Almirón Rejala,4.486209,Newcastle
Gabriel Martinelli Silva2023-12-09T17:30:00Z,Gabriel Martinelli Silva,4.105361,Arsenal
Anthony Gordon2023-12-10T16:30:00Z,Anthony Gordon,3.965232,Newcastle
Bukayo Saka2023-12-09T17:30:00Z,Bukayo Saka,3.882499,Arsenal
Phil Foden2023-12-10T14:00:00Z,Phil Foden,3.652689,Man City
Son Heung-min2023-12-10T16:30:00Z,Son Heung-min,3.487400,Spurs
Alejandro Garnacho2023-12-09T15:00:00Z,Alejandro Garnacho,3.413237,Man Utd


In [27]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Trent Alexander-Arnold2023-12-17T16:30:00Z,Trent Alexander-Arnold,4.432835,Liverpool
Kieran Trippier2023-12-16T15:00:00Z,Kieran Trippier,4.136633,Newcastle
Pedro Porro2023-12-15T20:00:00Z,Pedro Porro,3.533600,Spurs
Fabian Schär2023-12-16T15:00:00Z,Fabian Schär,3.531618,Newcastle
Virgil van Dijk2023-12-17T16:30:00Z,Virgil van Dijk,3.231536,Liverpool
Marcos Senesi2023-12-16T15:00:00Z,Marcos Senesi,3.209681,Bournemouth
Rúben Gato Alves Dias2023-12-16T15:00:00Z,Rúben Gato Alves Dias,3.129621,Man City
Adam Smith2023-12-16T15:00:00Z,Adam Smith,3.121610,Bournemouth
Pau Torres2023-12-17T14:00:00Z,Pau Torres,3.107906,Aston Villa


In [28]:
test[test["position"]=="GK"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Norberto Murara Neto2023-12-16T15:00:00Z,Norberto Murara Neto,3.510433,Bournemouth
Bernd Leno2023-12-16T15:00:00Z,Bernd Leno,3.492163,Fulham
Emiliano Martínez Romero2023-12-17T14:00:00Z,Emiliano Martínez Romero,3.458694,Aston Villa
Ederson Santana de Moraes2023-12-16T15:00:00Z,Ederson Santana de Moraes,3.406493,Man City
Mark Flekken2023-12-17T14:00:00Z,Mark Flekken,3.203621,Brentford
Jordan Pickford2023-12-16T17:30:00Z,Jordan Pickford,3.195132,Everton
James Trafford2023-12-16T17:30:00Z,James Trafford,3.172608,Burnley
Guglielmo Vicario2023-12-15T20:00:00Z,Guglielmo Vicario,3.150390,Spurs
Robert Sánchez2023-12-16T15:00:00Z,Robert Sánchez,3.098104,Chelsea


In [29]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Erling Haaland2023-12-16T15:00:00Z,Erling Haaland,4.166311,Man City
Ollie Watkins2023-12-17T14:00:00Z,Ollie Watkins,3.577323,Aston Villa
Julián Álvarez2023-12-16T15:00:00Z,Julián Álvarez,3.541544,Man City
Gabriel Fernando de Jesus2023-12-17T14:00:00Z,Gabriel Fernando de Jesus,3.462818,Arsenal
Alexander Isak2023-12-16T15:00:00Z,Alexander Isak,3.309576,Newcastle
Dominic Solanke2023-12-16T15:00:00Z,Dominic Solanke,3.231090,Bournemouth
Zeki Amdouni2023-12-16T17:30:00Z,Zeki Amdouni,2.968968,Burnley
Nicolas Jackson2023-12-16T15:00:00Z,Nicolas Jackson,2.944660,Chelsea
Raúl Jiménez2023-12-16T15:00:00Z,Raúl Jiménez,2.906443,Fulham


In [30]:
test["points"].sort_values(ascending=False).head(50)

index
Mohamed Salah2023-12-17T16:30:00Z                          4.910391
Trent Alexander-Arnold2023-12-17T16:30:00Z                 4.432835
Son Heung-min2023-12-15T20:00:00Z                          4.298387
Bukayo Saka2023-12-17T14:00:00Z                            4.220192
Erling Haaland2023-12-16T15:00:00Z                         4.166311
Kieran Trippier2023-12-16T15:00:00Z                        4.136633
Martin Ødegaard2023-12-17T14:00:00Z                        4.123751
Bruno Borges Fernandes2023-12-17T16:30:00Z                 3.778347
Bernardo Veiga de Carvalho e Silva2023-12-16T15:00:00Z     3.720655
Dejan Kulusevski2023-12-15T20:00:00Z                       3.717053
Phil Foden2023-12-16T15:00:00Z                             3.672477
Ollie Watkins2023-12-17T14:00:00Z                          3.577323
Vitaly Janelt2023-12-17T14:00:00Z                          3.553915
Julián Álvarez2023-12-16T15:00:00Z                         3.541544
Pedro Porro2023-12-15T20:00:00Z           

#Save predictions

In [31]:
test_copy0[test_copy0["position"]=="DEF"][["name","team","minutes"]].to_csv(f"predicted_dataset/lasso/GW{gameweek}/defenders_minutes.csv")
test_copy0[test_copy0["position"]=="GK"][["name","team","minutes"]].to_csv(f"predicted_dataset/lasso/GW{gameweek}/goalkeepers_minutes.csv")
test_copy0[test_copy0["position"]=="MID"][["name","team","minutes"]].to_csv(f"predicted_dataset/lasso/GW{gameweek}/midfielders_minutes.csv")
test_copy0[test_copy0["position"]=="FWD"][["name","team","minutes"]].to_csv(f"predicted_dataset/lasso/GW{gameweek}/forwards_minutes.csv")

In [32]:
test[test["position"]=="DEF"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/lasso/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GK"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/lasso/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/lasso/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].sort_values(by="points",ascending=False).to_csv(f"predicted_dataset/lasso/GW{gameweek}/forwards_points.csv")

In [18]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]]

,name,team,minutes
0,Fodé Ballo-Touré,Fulham,NaN
1,Timothy Castagne,Fulham,NaN
5,Ashley Phillips,Spurs,NaN
7,Rico Lewis,Man City,NaN
10,Rúben Gato Alves Dias,Man City,NaN
...,...,...,...
745,Luke Chambers,Liverpool,NaN
754,Reuell Walters,Arsenal,NaN
759,Tayo Adaramola,Crystal Palace,NaN
761,Valintino Adedokun,Brentford,NaN
